In [1]:
import cv2
print(cv2.__version__)


4.10.0


In [3]:
pip install mtcnn

  Using cached mtcnn-0.1.1-py3-none-any.whl.metadata (5.8 kB)
Using cached mtcnn-0.1.1-py3-none-any.whl (2.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow
print(tensorflow.__version__)


2.17.0


In [6]:
pip install keras-facenet


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10388 sha256=02a79a54c667e04e0fda95d92c0c7618540d21536215a0c8b19d8a64061acf97
  Stored in directory: c:\users\poulo\appdata\local\pip\cache\wheels\99\94\dd\cb1a65a7440ba6d508bd24346c15af0b1d24ff8b1cdb1c9959
Successfully built keras-facenet


In [39]:
import cv2
import numpy as np
import sqlite3
from keras_facenet import FaceNet
from mtcnn import MTCNN


In [40]:
detection_method = MTCNN()
embed_extract = FaceNet()

In [41]:
def capture_img():
    video_cap = cv2.VideoCapture(0)
    ret,frame = video_cap.read()
    video_cap.release()
    return frame

In [42]:
def detect_img(img):
    face = detection_method.detect_faces(img)
    return face

In [43]:
def ext_face(img,box):
    x,y,ht,wt = box
    face = img[y:y+ht,x:x+wt]
    face = cv2.resize(face,(160,160))
    face = face.astype('float32')/255
    return face
    

In [44]:
def face_embed(face_img):
    face_list = [face_img]
    embeddings = embed_extract.embeddings(face_list)
    return embed[0]

In [45]:
def embed_store(name, embedding):
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor() 
    c.execute('''CREATE TABLE IF NOT EXISTS faces (name TEXT, encoding BLOB)''') 
    c.execute("INSERT INTO faces (name, encoding) VALUES (?, ?)", (name, embedding.tobytes()))  
    conn.commit() 
    conn.close() 

In [51]:
def load_embed():
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute("Select name,encoding from faces") 
    rows = c.fetchall()
    verified_face_name = []
    verified_face_encoding = []
    for row in rows:
        name = row[0]
        encoding = row[1]
        encoding = np.frombuffer(encoding,dtype = np.float32)
        verified_face_name.append(name)
        verified_face_encoding.append(encoding)
    conn.close()
    return verified_face_name,verified_face_encoding
    
    

In [52]:
def calc_dist(embed1,embed2):
    return np.linalg.norm(embed1 - embed2)

In [18]:
def capture_store():
    image = capture_img()  # Capture an image from the webcam
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    faces = detect_img(img)  # Detect faces using MTCNN

    if len(faces) == 0:
        print("No face detected. Try again.")
        return

    # Process the first face detected (you can modify this to handle multiple faces)
    face = faces[0]
    box = face['box']  # Get bounding box of the detected face

    # Extract the face region from the image
    face_image = ext_face(img, box)

    # Generate the face embedding using FaceNet
    embedding = face_embed(face_img)

    # Prompt the user for a name
    name = input("Enter the name for the face: ")

    # Store the face embedding in the database
    embed_store(name, embedding)
    print(f"Face embedding for {name} has been stored in the database.")


#### All the peices together for the face detection and real-time verification system

1)	Import the required libraries:
2)	Initialize the pretrained models
3)	Capture the face image using the webcam (function 1)
4)	Detect the face using the MTCNN (function 2)
5)	Form a bounding box around the detected face and extract the face region (function 3)
6)	Generate face embeddings with Facenet from the extracted face region (function 4)
7)	Function to store the generated face embedding in the SQLite database(function 5)
8)	Load the embeddings from the database for verification (function 6)
9)	Calculate euclidean distance between stored and real time embedding(function 7)
10)	Capture and store a new face with their name during registration (function 8)
11)	Verify the user in real-time (function 9)


In [38]:
import cv2
import numpy as np
import sqlite3
from keras_facenet import FaceNet
from mtcnn import MTCNN

detection_method = MTCNN()
embed_extract = FaceNet()

def capture_img():
    video_cap = cv2.VideoCapture(0)
    ret,frame = video_cap.read()
    video_cap.release()
    return frame

def detect_img(img):
    face = detection_method.detect_faces(img)
    return face

def ext_face(img,box):
    x,y,wt,ht = box
    face = img[y:y+ht,x:x+wt]
    face = cv2.resize(face,(160,160))
    face = face.astype('float32')/255
    return face


def face_embed(face_img):
    face_list = [face_img]
    embeddings = embed_extract.embeddings(face_list)
    return embeddings[0]

def embed_store(name, embedding):
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor() 
    c.execute('''CREATE TABLE IF NOT EXISTS faces (name TEXT, encoding BLOB)''') 
    c.execute("INSERT INTO faces (name, encoding) VALUES (?, ?)", (name, embedding.tobytes()))  
    conn.commit() 
    conn.close() 
    
def load_embed():
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute("Select name,encoding from faces") 
    rows = c.fetchall()
    verified_face_name = []
    verified_face_encoding = []
    for row in rows:
        name = row[0]
        encoding = row[1]
        encoding = np.frombuffer(encoding,dtype = np.float32)
        verified_face_name.append(name)
        verified_face_encoding.append(encoding)
    conn.close()
    return verified_face_name,verified_face_encoding

def calc_dist(embed1,embed2):
    return np.linalg.norm(embed1 - embed2)

def capture_store():
    image = capture_img()  # Capture an image from the webcam
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    faces = detect_img(rgb_image)  # Detect faces using MTCNN

    if len(faces) == 0:
        print("No face detected. Try again.")
        return

   
    face = faces[0]
    box = face['box']  # Get bounding box of the detected face

    # Extract the face region from the image
    face_image = ext_face(rgb_image, box)

    # Generate the face embedding using FaceNet
    embedding = face_embed(face_image)

    
    name = input("Enter the name for the face: ")

    # Store the face embedding in the database
    embed_store(name, embedding)
    print(f"Face embedding for {name} has been stored in the database.")
    
    
    
def verify_face():
    image = capture_img()  # Capture an image from the webcam
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    faces = detect_img(rgb_image)  # Detect faces using MTCNN

    if len(faces) == 0:
        print("No face detected. Try again.")
        return

    
    face = faces[0]
    box = face['box']  # Get bounding box of the detected face

    # Extract the face region from the image
    face_image = ext_face(rgb_image, box)

    # Generate the face embedding using FaceNet
    embedding = face_embed(face_image)
    
    threshold = 0.5
    
    verified_face_name , verified_face_embedding = load_embed()
    
    for i , verified_user_embedding in enumerate(verified_face_embedding):
        dist = calc_dist(embedding,verified_user_embedding)
        if dist < threshold:
            print(f"Match found: {verified_face_name[i]} (distance: {dist:.4f})")
            return
    print("No Match found")

def main():
    print("Capture and store the face on registration..")
    capture_store()
    print("\n Verifying the face on real-time..")
    verify_face()
    
main()




Capture and store the face on registration..
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Enter the name for the face: Poulomi
Face embedding for Poulomi has been stored in the database.

 Verifying the face on real-time..
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [3]:
import cv2
import numpy as np
import sqlite3
from keras_facenet import FaceNet
from mtcnn import MTCNN

# Initialize MTCNN and FaceNet
detection_method = MTCNN()
embed_extract = FaceNet()

# Function to capture an image using the webcam
def capture_img():
    video_cap = cv2.VideoCapture(0)
    ret, frame = video_cap.read()
    video_cap.release()
    return frame

# Function to detect a face using MTCNN
def detect_img(img):
    face = detection_method.detect_faces(img)
    return face

# Function to extract the face region using bounding box from MTCNN
def ext_face(img, box):
    x, y, wt, ht = box
    face = img[y:y+ht, x:x+wt]
    face = cv2.resize(face, (160, 160))
    face = face.astype('float32') / 255
    return face

# Function to generate face embeddings using FaceNet
def face_embed(face_img):
    face_list = [face_img]
    embeddings = embed_extract.embeddings(face_list)
    return embeddings[0]

# Function to store face embedding and name in SQLite database
def embed_store(name, embedding):
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS faces (name TEXT, encoding BLOB)''') 
    c.execute("INSERT INTO faces (name, encoding) VALUES (?, ?)", (name, embedding.tobytes()))  
    conn.commit()
    conn.close()

# Function to load face embeddings from the database
def load_embed():
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute("Select name,encoding from faces")
    rows = c.fetchall()
    verified_face_name = []
    verified_face_encoding = []
    for row in rows:
        name = row[0]
        encoding = row[1]
        encoding = np.frombuffer(encoding, dtype=np.float32)
        verified_face_name.append(name)
        verified_face_encoding.append(encoding)
    conn.close()
    return verified_face_name, verified_face_encoding

# Function to calculate Euclidean distance between two embeddings
def calc_dist(embed1, embed2):
    return np.linalg.norm(embed1 - embed2)

# Function to capture and store face embedding in the database
def capture_store():
    image = capture_img()
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    faces = detect_img(rgb_image)
    if len(faces) == 0:
        print("No face detected. Try again.")
        return

    face = faces[0]
    box = face['box']
    face_image = ext_face(rgb_image, box)

    embedding = face_embed(face_image)

    name = input("Enter the name for the face: ")
    embed_store(name, embedding)
    print(f"Face embedding for {name} has been stored in the database.")

# Function to verify the captured face against stored embeddings
def verify_face():
    image = capture_img()
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    faces = detect_img(rgb_image)
    if len(faces) == 0:
        print("No face detected. Try again.")
        return False, None

    face = faces[0]
    box = face['box']
    face_image = ext_face(rgb_image, box)

    embedding = face_embed(face_image)

    threshold = 0.3  # Lower the threshold to make matching stricter
    verified_face_name, verified_face_embedding = load_embed()

    for i, verified_user_embedding in enumerate(verified_face_embedding):
        dist = calc_dist(embedding, verified_user_embedding)
        print(f"Distance from {verified_face_name[i]}: {dist:.4f}")  # Debug: print distance
        if dist < threshold:
            print(f"Match found: {verified_face_name[i]} (distance: {dist:.4f})")
            return True, verified_face_name[i]  # Return True for match
    print("No Match found")
    return False, None  # Return False for no match

# Function to evaluate the system accuracy based on user input
def evaluate_system():
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    total_tests = 0

    for i in range(5):  # Perform 5 verification tests
        print(f"\n--- Test {i + 1} ---")
        result, matched_name = verify_face()

        # Simulating the expected outcome by user input
        expected = input("Was this a correct match? (yes/no): ").strip().lower()

        total_tests += 1
        if expected == "yes" and result:
            true_positives += 1
        elif expected == "yes" and not result:
            false_negatives += 1
        elif expected == "no" and not result:
            true_negatives += 1
        elif expected == "no" and result:
            false_positives += 1

    # Calculate accuracy, precision, recall, and F1 score
    accuracy = (true_positives + true_negatives) / total_tests if total_tests > 0 else 0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1_score:.2f}")
    print(f"True Positives: {true_positives}, False Positives: {false_positives}")
    print(f"True Negatives: {true_negatives}, False Negatives: {false_negatives}")

# Main function to run the system
def main():
    print("Capture and store the face on registration..")
    capture_store()

    print("\nVerifying the face in real-time..")
    evaluate_system()

main()


Capture and store the face on registration..
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Enter the name for the face: Poulomi
Face embedding for Poulomi has been stored in the database.

Verifying the face in real-time..

--- Test 1 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━

In [9]:
import zipfile
import os

# Path to the uploaded zip file in Jupyter
zip_file_path = "./dataset.zip"

# Destination folder where you want to unzip the files
unzip_folder_path = "./dataset"

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_folder_path)

print(f"Dataset unzipped to: {unzip_folder_path}")


Dataset unzipped to: ./dataset


#### Performance evaluation of the entire actual system

In [5]:
import cv2
import numpy as np
import sqlite3
from keras_facenet import FaceNet
from mtcnn import MTCNN
import time
from datetime import datetime

# Initialize MTCNN and FaceNet
detection_method = MTCNN()
embed_extract = FaceNet()

# Function to capture an image using the webcam
def capture_img():
    video_cap = cv2.VideoCapture(0)
    ret, frame = video_cap.read()
    video_cap.release()
    return frame

# Function to detect a face using MTCNN
def detect_img(img):
    face = detection_method.detect_faces(img)
    return face

# Function to extract the face region using bounding box from MTCNN
def ext_face(img, box):
    x, y, wt, ht = box
    
    x = max(0, x)
    y = max(0, y)
    wt = min(wt, img.shape[1] - x)  
    ht = min(ht, img.shape[0] - y)  
    
    face = img[y:y+ht, x:x+wt]
    face = cv2.resize(face, (160, 160))
    face = face.astype('float32') / 255
    return face

# Function to generate face embeddings using FaceNet
def face_embed(face_img):
    face_list = [face_img]
    embeddings = embed_extract.embeddings(face_list)
    return embeddings[0]

# Function to store face embedding and name in SQLite database
def embed_store(name, embedding):
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS faces (name TEXT, encoding BLOB)''') 
    c.execute("INSERT INTO faces (name, encoding) VALUES (?, ?)", (name, embedding.tobytes()))  
    conn.commit()
    conn.close()

# Function to clear the database
def clear_database():
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS faces''')  # Drop the table if it exists
    conn.commit()
    conn.close()
    print("Database has been cleared.")

# Function to load face embeddings from the database
def load_embed():
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute("Select name,encoding from faces")
    rows = c.fetchall()
    verified_face_name = []
    verified_face_encoding = []
    for row in rows:
        name = row[0]
        encoding = row[1]
        encoding = np.frombuffer(encoding, dtype=np.float32)
        verified_face_name.append(name)
        verified_face_encoding.append(encoding)
    conn.close()
    return verified_face_name, verified_face_encoding

# Function to calculate Euclidean distance between two embeddings
def calc_dist(embed1, embed2):
    return np.linalg.norm(embed1 - embed2)

# Function to capture and store face embedding in the database
def capture_store():
    image = capture_img()
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    faces = detect_img(rgb_image)
    if len(faces) == 0:
        print("No face detected. Try again.")
        return

    face = faces[0]
    box = face['box']
    face_image = ext_face(rgb_image, box)

    embedding = face_embed(face_image)

    name = input("Enter the name for the face: ")
    embed_store(name, embedding)
    print(f"Face embedding for {name} has been stored in the database.")

# Function to verify the captured face against stored embeddings
def verify_face(threshold=0.3):
    image = capture_img()
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    faces = detect_img(rgb_image)
    if len(faces) == 0:
        print("No face detected. Try again.")
        return False, None

    face = faces[0]
    box = face['box']
    face_image = ext_face(rgb_image, box)

    embedding = face_embed(face_image)

    verified_face_name, verified_face_embedding = load_embed()

    for i, verified_user_embedding in enumerate(verified_face_embedding):
        dist = calc_dist(embedding, verified_user_embedding)
        if dist < threshold:
            print(f"Match found: {verified_face_name[i]} (distance: {dist:.4f})")
            return True, verified_face_name[i]  # Return True for match
    print("No Match found")
    return False, None  # Return False for no match

# Function to evaluate accuracy by testing both registered and unregistered faces
def evaluate_accuracy():
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    total_tests = 0

    # Test 5 known faces (True Positives)
    print("\n--- Testing True Positives ---")
    for i in range(5):
        print(f"Test {i + 1}: Show the system a face that has been registered.")
        
        # Print timestamp and delay for 5 seconds before capture
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Timestamp: {timestamp}. Get ready, capturing in 5 seconds...")
        time.sleep(5)  # 5-second delay for positioning
        
        result, _ = verify_face()

        if result:
            true_positives += 1
        else:
            false_negatives += 1

    # Test 5 unknown faces (True Negatives)
    print("\n--- Testing True Negatives ---")
    for i in range(5):
        print(f"Test {i + 1}: Show the system a face that has NOT been registered.")
        
        # Print timestamp and delay for 5 seconds before capture
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Timestamp: {timestamp}. Get ready, capturing in 5 seconds...")
        time.sleep(5)  # 5-second delay for positioning
        
        result, _ = verify_face()

        if not result:
            true_negatives += 1
        else:
            false_positives += 1

    # Calculate accuracy, precision, recall, and F1 score
    total_tests = true_positives + true_negatives + false_positives + false_negatives
    accuracy = (true_positives + true_negatives) / total_tests if total_tests > 0 else 0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1_score:.2f}")
    print(f"True Positives: {true_positives}, False Positives: {false_positives}")
    print(f"True Negatives: {true_negatives}, False Negatives: {false_negatives}")

# Main function to run the system
def main():
    print("Clearing the database before registration...")
    clear_database()  # Clear the database before registering new faces

    print("Capture and store the face on registration..")
    capture_store()

    print("\nVerifying the face with registered and unregistered users..")
    evaluate_accuracy()

main()


Clearing the database before registration...
Database has been cleared.
Capture and store the face on registration..
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Enter the name for the face: Poulomi
Face embedding for Poulomi has been stored in the database.

Verifying the face with registered and unregistered users..

--- Testing True Positives ---
Test 1: Show the system a face that has been registered.
Timestamp: 2024-09-11 22:36:26. Get ready, capturing in 5 seconds...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/s

In [9]:
import cv2
import numpy as np
import sqlite3
from keras_facenet import FaceNet
from mtcnn import MTCNN
import time
from datetime import datetime

# Initialize MTCNN and FaceNet
detection_method = MTCNN()
embed_extract = FaceNet()

# Function to capture an image using the webcam
def capture_img():
    video_cap = cv2.VideoCapture(0)
    ret, frame = video_cap.read()
    video_cap.release()
    return frame

# Function to detect a face using MTCNN
def detect_img(img):
    face = detection_method.detect_faces(img)
    return face

# Function to extract the face region using bounding box from MTCNN
def ext_face(img, box):
    x, y, wt, ht = box
    
    x = max(0, x)
    y = max(0, y)
    wt = min(wt, img.shape[1] - x)  
    ht = min(ht, img.shape[0] - y)  
    
    face = img[y:y+ht, x:x+wt]
    face = cv2.resize(face, (160, 160))
    face = face.astype('float32') / 255
    return face

# Function to generate face embeddings using FaceNet
def face_embed(face_img):
    face_list = [face_img]
    embeddings = embed_extract.embeddings(face_list)
    return embeddings[0]

# Function to store face embedding and name in SQLite database
def embed_store(name, embedding):
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS faces (name TEXT, encoding BLOB)''') 
    c.execute("INSERT INTO faces (name, encoding) VALUES (?, ?)", (name, embedding.tobytes()))  
    conn.commit()
    conn.close()

# Function to clear the database
def clear_database():
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS faces''')  # Drop the table if it exists
    conn.commit()
    conn.close()
    print("Database has been cleared.")

# Function to load face embeddings from the database
def load_embed():
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute("Select name,encoding from faces")
    rows = c.fetchall()
    verified_face_name = []
    verified_face_encoding = []
    for row in rows:
        name = row[0]
        encoding = row[1]
        encoding = np.frombuffer(encoding, dtype=np.float32)
        verified_face_name.append(name)
        verified_face_encoding.append(encoding)
    conn.close()
    return verified_face_name, verified_face_encoding

# Function to calculate Euclidean distance between two embeddings
def calc_dist(embed1, embed2):
    return np.linalg.norm(embed1 - embed2)

# Function to capture and store face embedding in the database
def capture_store():
    image = capture_img()
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    faces = detect_img(rgb_image)
    if len(faces) == 0:
        print("No face detected. Try again.")
        return

    face = faces[0]
    box = face['box']
    face_image = ext_face(rgb_image, box)

    embedding = face_embed(face_image)

    name = input("Enter the name for the face: ")
    embed_store(name, embedding)
    print(f"Face embedding for {name} has been stored in the database.")

# Function to verify the captured face against stored embeddings
def verify_face(threshold=0.3):
    image = capture_img()
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    faces = detect_img(rgb_image)
    if len(faces) == 0:
        print("No face detected. Try again.")
        return False, None

    face = faces[0]
    box = face['box']
    face_image = ext_face(rgb_image, box)

    embedding = face_embed(face_image)

    verified_face_name, verified_face_embedding = load_embed()

    for i, verified_user_embedding in enumerate(verified_face_embedding):
        dist = calc_dist(embedding, verified_user_embedding)
        if dist < threshold:
            print(f"Match found: {verified_face_name[i]} (distance: {dist:.4f})")
            return True, verified_face_name[i]  # Return True for match
    print("No Match found")
    return False, None  # Return False for no match

# Function to evaluate accuracy by testing both registered and unregistered faces
def evaluate_accuracy():
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    total_tests = 0

    # Test 5 known faces (True Positives)
    print("\n--- Testing True Positives ---")
    for i in range(5):
        print(f"Test {i + 1}: Show the system a face that has been registered.")
        
        # Print timestamp and delay for 5 seconds before capture
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Timestamp: {timestamp}. Get ready, capturing in 5 seconds...")
        time.sleep(5)  # 5-second delay for positioning
        
        result, _ = verify_face()

        if result:
            true_positives += 1
        else:
            false_negatives += 1

    # Test 5 unknown faces (True Negatives)
    print("\n--- Testing True Negatives ---")
    for i in range(5):
        print(f"Test {i + 1}: Show the system a face that has NOT been registered.")
        
        # Print timestamp and delay for 5 seconds before capture
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Timestamp: {timestamp}. Get ready, capturing in 5 seconds...")
        time.sleep(5)  # 5-second delay for positioning
        
        result, _ = verify_face()

        if not result:
            true_negatives += 1
        else:
            false_positives += 1

    # Calculate accuracy, precision, recall, and F1 score
    total_tests = true_positives + true_negatives + false_positives + false_negatives
    accuracy = (true_positives + true_negatives) / total_tests if total_tests > 0 else 0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1_score:.2f}")
    print(f"True Positives: {true_positives}, False Positives: {false_positives}")
    print(f"True Negatives: {true_negatives}, False Negatives: {false_negatives}")

# Main function to run the system
def main():
    print("Clearing the database before registration...")
    clear_database()  # Clear the database before registering new faces

    print("Capture and store the face on registration..")
    capture_store()

    print("\nVerifying the face with registered and unregistered users..")
    evaluate_accuracy()

main()


Clearing the database before registration...
Database has been cleared.
Capture and store the face on registration..
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Enter the name for the face: Poulomi
Face embedding for Poulomi has been stored in the database.

Verifying the face with registered and unregistered users..

--- Testing True Positives ---
Test 1: Show the system a face that has been registered.
Timestamp: 2024-09-11 22:48:54. Get ready, capturing in 5 seconds...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/

In [10]:
import cv2
import numpy as np
import sqlite3
from keras_facenet import FaceNet
from mtcnn import MTCNN
import time
from datetime import datetime

# Initialize MTCNN and FaceNet
detection_method = MTCNN()
embed_extract = FaceNet()

# Function to capture an image using the webcam
def capture_img():
    video_cap = cv2.VideoCapture(0)
    ret, frame = video_cap.read()
    video_cap.release()
    return frame

# Function to detect a face using MTCNN
def detect_img(img):
    face = detection_method.detect_faces(img)
    return face

# Function to extract the face region using bounding box from MTCNN
def ext_face(img, box):
    x, y, wt, ht = box
    
    x = max(0, x)
    y = max(0, y)
    wt = min(wt, img.shape[1] - x)  
    ht = min(ht, img.shape[0] - y)  
    
    face = img[y:y+ht, x:x+wt]
    face = cv2.resize(face, (160, 160))
    face = face.astype('float32') / 255
    return face

# Function to generate face embeddings using FaceNet
def face_embed(face_img):
    face_list = [face_img]
    embeddings = embed_extract.embeddings(face_list)
    return embeddings[0]

# Function to store face embedding and name in SQLite database
def embed_store(name, embedding):
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS faces (name TEXT, encoding BLOB)''') 
    c.execute("INSERT INTO faces (name, encoding) VALUES (?, ?)", (name, embedding.tobytes()))  
    conn.commit()
    conn.close()

# Function to clear the database
def clear_database():
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS faces''')  # Drop the table if it exists
    conn.commit()
    conn.close()
    print("Database has been cleared.")

# Function to load face embeddings from the database
def load_embed():
    conn = sqlite3.connect('face_recognition.db')  
    c = conn.cursor()
    c.execute("Select name,encoding from faces")
    rows = c.fetchall()
    verified_face_name = []
    verified_face_encoding = []
    for row in rows:
        name = row[0]
        encoding = row[1]
        encoding = np.frombuffer(encoding, dtype=np.float32)
        verified_face_name.append(name)
        verified_face_encoding.append(encoding)
    conn.close()
    return verified_face_name, verified_face_encoding

# Function to calculate Euclidean distance between two embeddings
def calc_dist(embed1, embed2):
    return np.linalg.norm(embed1 - embed2)

# Function to capture and store face embedding in the database
def capture_store():
    image = capture_img()
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    faces = detect_img(rgb_image)
    if len(faces) == 0:
        print("No face detected. Try again.")
        return

    face = faces[0]
    box = face['box']
    face_image = ext_face(rgb_image, box)

    embedding = face_embed(face_image)

    name = input("Enter the name for the face: ")
    embed_store(name, embedding)
    print(f"Face embedding for {name} has been stored in the database.")

# Function to verify the captured face against stored embeddings
def verify_face(threshold=0.3):
    image = capture_img()
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    faces = detect_img(rgb_image)
    if len(faces) == 0:
        print("No face detected. Try again.")
        return False, None

    face = faces[0]
    box = face['box']
    face_image = ext_face(rgb_image, box)

    embedding = face_embed(face_image)

    verified_face_name, verified_face_embedding = load_embed()

    for i, verified_user_embedding in enumerate(verified_face_embedding):
        dist = calc_dist(embedding, verified_user_embedding)
        if dist < threshold:
            print(f"Match found: {verified_face_name[i]} (distance: {dist:.4f})")
            return True, verified_face_name[i]  # Return True for match
    print("No Match found")
    return False, None  # Return False for no match

# Function to evaluate accuracy by testing both registered and unregistered faces
def evaluate_accuracy():
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    total_tests = 0

    # Test 5 known faces (True Positives)
    print("\n--- Testing True Positives ---")
    for i in range(5):
        print(f"Test {i + 1}: Show the system a face that has been registered.")
        
        
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Timestamp: {timestamp}. Get ready, capturing in 5 seconds...")
        time.sleep(5) 
        
        result, _ = verify_face()

        if result:
            true_positives += 1
        else:
            false_negatives += 1

    # Test 5 unknown faces (True Negatives)
    print("\n--- Testing True Negatives ---")
    for i in range(5):
        print(f"Test {i + 1}: Show the system a face that has NOT been registered.")
        
        # Print timestamp and delay for 5 seconds before capture
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Timestamp: {timestamp}. Get ready, capturing in 5 seconds...")
        time.sleep(5)  # 5-second delay for positioning
        
        result, _ = verify_face()

        if not result:
            true_negatives += 1
        else:
            false_positives += 1

    # Calculate accuracy, precision, recall, and F1 score
    total_tests = true_positives + true_negatives + false_positives + false_negatives
    accuracy = (true_positives + true_negatives) / total_tests if total_tests > 0 else 0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1_score:.2f}")
    print(f"True Positives: {true_positives}, False Positives: {false_positives}")
    print(f"True Negatives: {true_negatives}, False Negatives: {false_negatives}")

# Main function to run the system
def main():
    print("Clearing the database before registration...")
    clear_database()  

    print("Capture and store the face on registration..")
    capture_store()

    print("\nVerifying the face with registered and unregistered users..")
    evaluate_accuracy()

main()


Clearing the database before registration...
Database has been cleared.
Capture and store the face on registration..
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Enter the name for the face: Poulomi
Face embedding for Poulomi has been stored in the database.

Verifying the face with registered and unregistered users..

--- Testing True Positives ---
Test 1: Show the system a face that has been registered.
Timestamp: 2024-09-11 22:58:11. Get ready, capturing in 5 seconds...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/